In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

class HierarchicalFireEventClustering:
    def __init__(self, project_id, dataset_id):
        self.client = bigquery.Client(project=project_id)
        self.dataset_id = dataset_id
        self.project_id = project_id

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """Calculate haversine distance between two points in kilometers"""
        R = 6371  # earthss radius in kilometers
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        return R * c

    def define_geographic_regions(self):
        """Define geographic regions"""
        regions = {
            'Pacific_West': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -125.0, 'lon_max': -115.0},
            'Mountain_West': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -115.0, 'lon_max': -105.0},
            'Great_Plains': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -105.0, 'lon_max': -95.0},
            'South_Texas': {'lat_min': 25.0, 'lat_max': 32.0, 'lon_min': -105.0, 'lon_max': -95.0},
            'South_Central': {'lat_min': 25.0, 'lat_max': 37.0, 'lon_min': -95.0, 'lon_max': -85.0},
            'Midwest': {'lat_min': 37.0, 'lat_max': 49.0, 'lon_min': -95.0, 'lon_max': -85.0},
            'Southeast': {'lat_min': 25.0, 'lat_max': 37.0, 'lon_min': -85.0, 'lon_max': -75.0},
            'Northeast': {'lat_min': 37.0, 'lat_max': 49.0, 'lon_min': -85.0, 'lon_max': -67.0},
            'Florida': {'lat_min': 25.0, 'lat_max': 32.0, 'lon_min': -85.0, 'lon_max': -80.0}
        }
        return regions

    def get_region_parameters(self, region_name):
        """Get parameters with proper radians conversion"""

        cropland_regions = ['Great_Plains', 'Midwest', 'South_Central']

        region_params_km = {
            'Pacific_West': {'spatial_km': 5.0, 'temporal_days': 5, 'min_samples': 5},
            'Mountain_West': {'spatial_km': 5.0, 'temporal_days': 5, 'min_samples': 5},
            'Great_Plains': {'spatial_km': 3.0, 'temporal_days': 5, 'min_samples': 6},      # Reduced from 8
            'South_Texas': {'spatial_km': 3.0, 'temporal_days': 5, 'min_samples': 5},
            'South_Central': {'spatial_km': 2.0, 'temporal_days': 5, 'min_samples': 7},     # Reduced from 10
            'Midwest': {'spatial_km': 3.0, 'temporal_days': 5, 'min_samples': 6},          # Reduced from 8
            'Southeast': {'spatial_km': 2.0, 'temporal_days': 5, 'min_samples': 5},
            'Northeast': {'spatial_km': 5.0, 'temporal_days': 5, 'min_samples': 5},
            'Florida': {'spatial_km': 2.0, 'temporal_days': 5, 'min_samples': 5}
        }

        params_km = region_params_km.get(region_name,
                                       {'spatial_km': 5.0, 'temporal_days': 3, 'min_samples': 5})

        # convert to radians
        spatial_degrees = params_km['spatial_km'] / 111.0
        spatial_radians = np.radians(spatial_degrees)

        return {
            'spatial_eps_radians': spatial_radians,
            'spatial_km': params_km['spatial_km'],
            'temporal_days': params_km['temporal_days'],
            'min_samples': params_km['min_samples']
        }

    def _get_cluster_extent(self, cluster_data):
        """Calculate spatial extent of cluster in km"""
        min_lat = cluster_data['latitude'].min()
        max_lat = cluster_data['latitude'].max()
        min_lon = cluster_data['longitude'].min()
        max_lon = cluster_data['longitude'].max()

        return self.haversine_distance(min_lat, min_lon, max_lat, max_lon)

    def _break_up_large_cluster(self, cluster_data, max_size_km=25):
        """Try to break up large clusters using tighter clustering"""

        initial_extent = self._get_cluster_extent(cluster_data)
        print(f"       Attempting to break up {initial_extent:.1f}km cluster...")

        # try progressively tighter clustering
        sub_clusters = []

        # Tighter spatial clustering
        coords = cluster_data[['longitude', 'latitude']].values
        coords_rad = np.radians(coords)

        #  tighter spatial
        tight_spatial_eps = np.radians(2.0 / 111.0)  # 2km radius instead of 1km

        dbscan_tight = DBSCAN(
            eps=tight_spatial_eps,
            min_samples=4,  # lower min_samples for sub-clustering
            metric='haversine'
        )

        tight_labels = dbscan_tight.fit_predict(coords_rad)
        cluster_data['tight_spatial_cluster'] = tight_labels

        # temporal refinement on each tight spatial cluster
        event_id = 0
        for tight_cluster in cluster_data['tight_spatial_cluster'].unique():
            if tight_cluster == -1:  # skip noise
                continue

            tight_cluster_data = cluster_data[cluster_data['tight_spatial_cluster'] == tight_cluster].copy()

            # apply temporal splitting with 3-day gaps
            tight_cluster_data = tight_cluster_data.sort_values('fire_date')
            tight_cluster_data['temp_group'] = 0
            current_group = 0

            for i in range(1, len(tight_cluster_data)):
                prev_date = tight_cluster_data.iloc[i-1]['fire_date']
                curr_date = tight_cluster_data.iloc[i]['fire_date']

                if (curr_date - prev_date).days > 5:  # 5-day gap instead of 3
                    current_group += 1

                tight_cluster_data.iloc[i, tight_cluster_data.columns.get_loc('temp_group')] = current_group

            # create sub-events
            for temp_group in tight_cluster_data['temp_group'].unique():
                sub_event_data = tight_cluster_data[tight_cluster_data['temp_group'] == temp_group].copy()

                # check if sub-event is reasonable size
                sub_extent = self._get_cluster_extent(sub_event_data)
                if sub_extent <= max_size_km:
                    sub_event_data['fire_event_id'] = event_id
                    sub_event_data = sub_event_data.drop(['tight_spatial_cluster', 'temp_group'], axis=1)
                    sub_clusters.append(sub_event_data)
                    event_id += 1
                else:
                    print(f"         Sub-cluster still too large ({sub_extent:.1f}km), skipping")

        if sub_clusters:
            result = pd.concat(sub_clusters, ignore_index=True)
            n_sub_events = len(result['fire_event_id'].unique())
            print(f"         Successfully broke into {n_sub_events} sub-events")
            return result
        else:
            print(f"         Could not break up cluster, treating as noise")
            return pd.DataFrame()

    def _cluster_region_data_hierarchical(self, df, params):
        """Apply hierarchical clustering with breakup logic"""

        # initial spatial clustering
        coords = df[['longitude', 'latitude']].values
        coords_rad = np.radians(coords)

        dbscan = DBSCAN(
            eps=params['spatial_eps_radians'],
            min_samples=params['min_samples'],
            metric='haversine'
        )
        spatial_labels = dbscan.fit_predict(coords_rad)
        df['spatial_cluster'] = spatial_labels

        n_spatial_clusters = len(set(spatial_labels)) - (1 if -1 in spatial_labels else 0)
        print(f"     Initial spatial clusters: {n_spatial_clusters}")

        # process each spatial cluster
        fire_events = []
        event_id = 0

        for spatial_cluster in df['spatial_cluster'].unique():
            if spatial_cluster == -1:
                continue

            cluster_data = df[df['spatial_cluster'] == spatial_cluster].copy()
            cluster_data = cluster_data.sort_values('fire_date')

            # apply temporal splitting
            cluster_data['temp_group'] = 0
            current_group = 0

            for i in range(1, len(cluster_data)):
                prev_date = cluster_data.iloc[i-1]['fire_date']
                curr_date = cluster_data.iloc[i]['fire_date']

                if (curr_date - prev_date).days > params['temporal_days']:
                    current_group += 1

                cluster_data.iloc[i, cluster_data.columns.get_loc('temp_group')] = current_group

            # process each temporal group
            for temp_group in cluster_data['temp_group'].unique():
                group_data = cluster_data[cluster_data['temp_group'] == temp_group].copy()
                extent_km = self._get_cluster_extent(group_data)

                # apply hierarchical logic
                if extent_km <= 25:
                    # Size OK - accept as is?
                    group_data['fire_event_id'] = event_id
                    group_data = group_data.drop(['spatial_cluster', 'temp_group'], axis=1)
                    fire_events.append(group_data)
                    event_id += 1

                elif extent_km <= 75:  # increased from 50km
                    # size concerning but salvageable - try to break up
                    print(f"     Large cluster detected: {extent_km:.1f}km")
                    sub_events = self._break_up_large_cluster(group_data, max_size_km=30)  # allow 30km sub-events

                    if len(sub_events) > 0:
                        # adjust event IDs
                        max_sub_id = sub_events['fire_event_id'].max()
                        sub_events['fire_event_id'] += event_id
                        fire_events.append(sub_events)
                        event_id += max_sub_id + 1
                    # ff break up fails, cluster is discarded (treated as noise)

                else:
                    # size too large - hard reject
                    print(f"     Rejected oversized cluster: {extent_km:.1f}km (>75km hard limit)")

        if fire_events:
            result_df = pd.concat(fire_events, ignore_index=True)
            print(f"     Final fire events: {len(result_df['fire_event_id'].unique())}")
            return result_df
        else:
            return pd.DataFrame()

    def process_geographic_region(self, year, region_name, region_bounds):
        """Process fire events for a specific geographic region"""

        params = self.get_region_parameters(region_name)
        table_name = f"emission_{year}"

        print(f"\nProcessing region: {region_name}")
        print(f"   Bounds: {region_bounds}")
        print(f"   Parameters: {params['spatial_km']:.1f}km, temporal={params['temporal_days']}d, min_samples={params['min_samples']}")

        query = f"""
        SELECT
            id, year, doy, longitude, latitude, fire_date,
            grid10k, covertype, fuelcode, area_burned,
            consumed_fuel, ECO2, burn_source, burnday_source
        FROM `{self.project_id}.{self.dataset_id}.{table_name}`
        WHERE longitude IS NOT NULL
        AND latitude IS NOT NULL
        AND fire_date IS NOT NULL
        AND latitude >= {region_bounds['lat_min']}
        AND latitude < {region_bounds['lat_max']}
        AND longitude >= {region_bounds['lon_min']}
        AND longitude < {region_bounds['lon_max']}
        ORDER BY fire_date, longitude, latitude
        """

        df = self.client.query(query).to_dataframe()

        if len(df) == 0:
            print(f"   No data found for {region_name}")
            return pd.DataFrame()

        print(f"   Found {len(df):,} records")
        df['fire_date'] = pd.to_datetime(df['fire_date'])

        clustered_df = self._cluster_region_data_hierarchical(df, params)

        if len(clustered_df) > 0:
            clustered_df['region'] = region_name
            unique_events = len(clustered_df['fire_event_id'].unique())
            print(f"   Found {unique_events} fire events in {region_name}")
            return clustered_df
        else:
            print(f"   No fire events found in {region_name}")
            return pd.DataFrame()

    def process_year_hierarchical(self, year):
        """Process entire year using hierarchical clustering"""

        print(f"HIERARCHICAL FIRE CLUSTERING for {year}")
        print("Using intelligent cluster breakup:")
        print("  - ≤25km: Accept as-is")
        print("  - 25-75km: Try to break up (↑ increased range)")
        print("  - >75km: Hard reject")
        print("  - 5-day temporal gaps (↑ increased)")
        print("  - Balanced min_samples (↓ reduced)")

        regions = self.define_geographic_regions()
        print("=" * 80)

        # total count
        total_query = f"""
        SELECT COUNT(*) as total_count
        FROM `{self.project_id}.{self.dataset_id}.emission_{year}`
        WHERE longitude IS NOT NULL
        AND latitude IS NOT NULL
        AND fire_date IS NOT NULL
        """

        total_result = self.client.query(total_query).to_dataframe()
        original_count = total_result['total_count'].iloc[0]

        all_regional_events = []
        global_event_id = 0

        for region_name, region_bounds in regions.items():
            regional_events = self.process_geographic_region(year, region_name, region_bounds)

            if len(regional_events) > 0:
                # adjust event IDs
                max_regional_id = regional_events['fire_event_id'].max()
                regional_events['fire_event_id'] += global_event_id
                global_event_id += max_regional_id + 1

                all_regional_events.append(regional_events)

        if all_regional_events:
            combined_events = pd.concat(all_regional_events, ignore_index=True)

            # remove dups
            initial_count = len(combined_events)
            combined_events = combined_events.drop_duplicates(subset=['id'], keep='first')
            final_count = len(combined_events)

            if initial_count != final_count:
                duplicate_count = initial_count - final_count
                print(f"WARNING: Removed {duplicate_count} duplicate records ({duplicate_count/initial_count*100:.1f}%)")

            print(f"\nHIERARCHICAL CLUSTERING RESULTS:")
            print(f"Total fire events: {len(combined_events['fire_event_id'].unique())}")
            print(f"Total data points: {len(combined_events):,}")
            print(f"Original data points: {original_count:,}")
            print(f"Coverage: {len(combined_events)/original_count*100:.1f}% of original data")

            return combined_events
        else:
            return pd.DataFrame()

    def validate_results(self, fire_events_df):
        """Validate hierarchical clustering results"""

        if len(fire_events_df) == 0:
            print("No fire events to validate")
            return pd.DataFrame()

        # calc stats
        event_stats = fire_events_df.groupby('fire_event_id').agg({
            'fire_date': ['min', 'max'],
            'longitude': ['min', 'max'],
            'latitude': ['min', 'max'],
            'id': 'count'
        }).reset_index()

        event_stats.columns = ['fire_event_id', 'start_date', 'end_date',
                              'min_lon', 'max_lon', 'min_lat', 'max_lat', 'point_count']

        event_stats['duration_days'] = (event_stats['end_date'] - event_stats['start_date']).dt.days
        event_stats['spatial_extent_km'] = event_stats.apply(
            lambda row: self.haversine_distance(row['min_lat'], row['min_lon'],
                                              row['max_lat'], row['max_lon']), axis=1
        )

        # quality checks
        flagged_size = event_stats[event_stats['spatial_extent_km'] > 30]
        hard_fails = event_stats[event_stats['spatial_extent_km'] > 75]

        print(f"\nVALIDATION RESULTS:")
        print(f"Total events: {len(event_stats)}")
        print(f"Average spatial extent: {event_stats['spatial_extent_km'].mean():.1f} km")
        print(f"Average duration: {event_stats['duration_days'].mean():.1f} days")
        print(f"Max spatial extent: {event_stats['spatial_extent_km'].max():.1f} km")
        print(f"Max duration: {event_stats['duration_days'].max():.1f} days")
        print(f"\nQUALITY CHECK:")
        print(f"Events >30km (flagged): {len(flagged_size)} ({len(flagged_size)/len(event_stats)*100:.1f}%)")
        print(f"Events >75km (hard fails): {len(hard_fails)} (should be 0)")

        if len(flagged_size) > 0:
            print(f"\nFLAGGED EVENTS (30-75km):")
            for _, row in flagged_size.head(10).iterrows():
                print(f"  Event {row['fire_event_id']}: {row['spatial_extent_km']:.1f}km, "
                      f"{row['duration_days']}d, {row['point_count']} points")

        if len(hard_fails) > 0:
            print(f"\nHARD FAILURES (>75km):")
            for _, row in hard_fails.iterrows():
                print(f"  Event {row['fire_event_id']}: {row['spatial_extent_km']:.1f}km, "
                      f"{row['duration_days']}d, {row['point_count']} points")

        return event_stats

    def save_results_to_csv(self, fire_events_df, filename):
        """Save results to CSV file"""
        fire_events_df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")

# use
def main():
    print("HIERARCHICAL FIRE CLUSTERING")
    print("Intelligently breaks up large clusters instead of discarding them")
    print("Based on the recommended thresholds:")
    print("  - Hard size cap: 50km")
    print("  - QA flag threshold: 25km")
    print("  - Temporal gap: 3 days")
    print("  - Min samples: 8-10 in cropland/grassland, 5 elsewhere")
    print()

    clustering = HierarchicalFireEventClustering(
        project_id="code-for-planet",
        dataset_id="emission_db"
    )

    year = 2004
    fire_events = clustering.process_year_hierarchical(year=year)

    if len(fire_events) > 0:
        stats = clustering.validate_results(fire_events)

        clustering.save_results_to_csv(fire_events, f"fire_events_{year}_hierarchical.csv")
        clustering.save_results_to_csv(stats, f"fire_event_stats_{year}_hierarchical.csv")

        print(f"\nResults saved:")
        print(f"- fire_events_{year}_hierarchical.csv")
        print(f"- fire_event_stats_{year}_hierarchical.csv")

        if len(stats) > 0:
            print(f"\nResults by region:")
            fire_events_with_stats = fire_events.merge(
                stats[['fire_event_id', 'spatial_extent_km', 'duration_days']],
                on='fire_event_id'
            )
            region_summary = fire_events_with_stats.groupby('region').agg({
                'fire_event_id': 'nunique',
                'spatial_extent_km': 'mean',
                'duration_days': 'mean'
            }).round(1)
            print(region_summary)
    else:
        print("No fire events found")

if __name__ == "__main__":
    main()

HIERARCHICAL FIRE CLUSTERING
Intelligently breaks up large clusters instead of discarding them
Based on the recommended thresholds:
  - Hard size cap: 50km
  - QA flag threshold: 25km
  - Temporal gap: 3 days
  - Min samples: 8-10 in cropland/grassland, 5 elsewhere

HIERARCHICAL FIRE CLUSTERING for 2004
Using intelligent cluster breakup:
  - ≤25km: Accept as-is
  - 25-75km: Try to break up (↑ increased range)
  - >75km: Hard reject
  - 5-day temporal gaps (↑ increased)
  - Balanced min_samples (↓ reduced)

Processing region: Pacific_West
   Bounds: {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -125.0, 'lon_max': -115.0}
   Parameters: 5.0km, temporal=5d, min_samples=5
   Found 41,845 records
     Initial spatial clusters: 418
     Large cluster detected: 27.7km
       Attempting to break up 27.7km cluster...
         Successfully broke into 5 sub-events
     Large cluster detected: 26.3km
       Attempting to break up 26.3km cluster...
         Successfully broke into 4 sub-events
    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

class DiagnosticFireClustering:
    def __init__(self, project_id, dataset_id):
        self.client = bigquery.Client(project=project_id)
        self.dataset_id = dataset_id
        self.project_id = project_id
        self.data_loss_tracking = {}

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """Calculate haversine distance between two points in kilometers"""
        R = 6371
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        return R * c

    def get_region_parameters(self, region_name):
        """Get parameters with proper radians conversion"""

        # much more permissive parameters to reduce data loss
        region_params_km = {
            'Pacific_West': {'spatial_km': 8.0, 'temporal_days': 7, 'min_samples': 4},
            'Mountain_West': {'spatial_km': 8.0, 'temporal_days': 7, 'min_samples': 4},
            'Great_Plains': {'spatial_km': 5.0, 'temporal_days': 7, 'min_samples': 4},
            'South_Texas': {'spatial_km': 5.0, 'temporal_days': 7, 'min_samples': 4},
            'South_Central': {'spatial_km': 3.0, 'temporal_days': 7, 'min_samples': 4},
            'Midwest': {'spatial_km': 5.0, 'temporal_days': 7, 'min_samples': 4},
            'Southeast': {'spatial_km': 3.0, 'temporal_days': 7, 'min_samples': 4},
            'Northeast': {'spatial_km': 8.0, 'temporal_days': 7, 'min_samples': 4},
            'Florida': {'spatial_km': 3.0, 'temporal_days': 7, 'min_samples': 4}
        }

        params_km = region_params_km.get(region_name,
                                       {'spatial_km': 5.0, 'temporal_days': 7, 'min_samples': 4})

        # convert to radians
        spatial_degrees = params_km['spatial_km'] / 111.0
        spatial_radians = np.radians(spatial_degrees)

        return {
            'spatial_eps_radians': spatial_radians,
            'spatial_km': params_km['spatial_km'],
            'temporal_days': params_km['temporal_days'],
            'min_samples': params_km['min_samples']
        }

    def _get_cluster_extent(self, cluster_data):
        """Calculate spatial extent of cluster in km"""
        min_lat = cluster_data['latitude'].min()
        max_lat = cluster_data['latitude'].max()
        min_lon = cluster_data['longitude'].min()
        max_lon = cluster_data['longitude'].max()

        return self.haversine_distance(min_lat, min_lon, max_lat, max_lon)

    def _track_data_loss(self, stage, region, original_count, remaining_count, notes=""):
        """Track data loss at each stage"""
        if region not in self.data_loss_tracking:
            self.data_loss_tracking[region] = []

        lost_count = original_count - remaining_count
        lost_percent = (lost_count / original_count * 100) if original_count > 0 else 0

        self.data_loss_tracking[region].append({
            'stage': stage,
            'original': original_count,
            'remaining': remaining_count,
            'lost': lost_count,
            'lost_percent': lost_percent,
            'notes': notes
        })

    def _simple_size_filter(self, group_data, max_size_km=100):
        """Simple size filter - just check if cluster is reasonable"""
        extent_km = self._get_cluster_extent(group_data)

        if extent_km <= max_size_km:
            return group_data, True, f"accepted ({extent_km:.1f}km)"
        else:
            return pd.DataFrame(), False, f"rejected ({extent_km:.1f}km > {max_size_km}km)"

    def _cluster_region_data_diagnostic(self, df, params, region_name):
        """Apply clustering with detailed diagnostics"""

        original_count = len(df)
        print(f"     Starting with {original_count:,} records")

        coords = df[['longitude', 'latitude']].values
        coords_rad = np.radians(coords)

        dbscan = DBSCAN(
            eps=params['spatial_eps_radians'],
            min_samples=params['min_samples'],
            metric='haversine'
        )
        spatial_labels = dbscan.fit_predict(coords_rad)
        df['spatial_cluster'] = spatial_labels

        noise_count = np.sum(spatial_labels == -1)
        clustered_count = original_count - noise_count
        n_spatial_clusters = len(set(spatial_labels)) - (1 if -1 in spatial_labels else 0)

        print(f"     Initial clustering: {n_spatial_clusters} clusters, {noise_count:,} noise points")
        self._track_data_loss("initial_clustering", region_name, original_count, clustered_count,
                             f"{n_spatial_clusters} clusters, {noise_count} noise")

        fire_events = []
        event_id = 0
        temporal_processed = 0
        temporal_accepted = 0

        for spatial_cluster in df['spatial_cluster'].unique():
            if spatial_cluster == -1:
                continue

            cluster_data = df[df['spatial_cluster'] == spatial_cluster].copy()
            cluster_data = cluster_data.sort_values('fire_date')
            temporal_processed += len(cluster_data)

            cluster_data['temp_group'] = 0
            current_group = 0

            for i in range(1, len(cluster_data)):
                prev_date = cluster_data.iloc[i-1]['fire_date']
                curr_date = cluster_data.iloc[i]['fire_date']

                if (curr_date - prev_date).days > params['temporal_days']:
                    current_group += 1

                cluster_data.iloc[i, cluster_data.columns.get_loc('temp_group')] = current_group

            for temp_group in cluster_data['temp_group'].unique():
                group_data = cluster_data[cluster_data['temp_group'] == temp_group].copy()

                filtered_data, accepted, notes = self._simple_size_filter(group_data, max_size_km=100)

                if accepted:
                    filtered_data['fire_event_id'] = event_id
                    filtered_data = filtered_data.drop(['spatial_cluster', 'temp_group'], axis=1)
                    fire_events.append(filtered_data)
                    temporal_accepted += len(filtered_data)
                    event_id += 1

        print(f"     Temporal processing: {temporal_processed:,} → {temporal_accepted:,} points")
        self._track_data_loss("temporal_and_size", region_name, temporal_processed, temporal_accepted,
                             f"after temporal + size filtering")

        if fire_events:
            result_df = pd.concat(fire_events, ignore_index=True)
            final_count = len(result_df)
            n_events = len(result_df['fire_event_id'].unique())
            print(f"     Final result: {n_events} events, {final_count:,} points")
            self._track_data_loss("final", region_name, original_count, final_count, f"{n_events} events")
            return result_df
        else:
            print(f"     Final result: 0 events, 0 points")
            self._track_data_loss("final", region_name, original_count, 0, "no events")
            return pd.DataFrame()

    def process_region_diagnostic(self, year, region_name, region_bounds):
        """Process region with detailed diagnostics"""

        params = self.get_region_parameters(region_name)
        table_name = f"emission_{year}"

        print(f"\n{'='*60}")
        print(f"PROCESSING REGION: {region_name}")
        print(f"Bounds: {region_bounds}")
        print(f"Parameters: {params['spatial_km']:.1f}km, {params['temporal_days']}d, min_samples={params['min_samples']}")

        query = f"""
        SELECT
            id, year, doy, longitude, latitude, fire_date,
            grid10k, covertype, fuelcode, area_burned,
            consumed_fuel, ECO2, burn_source, burnday_source
        FROM `{self.project_id}.{self.dataset_id}.{table_name}`
        WHERE longitude IS NOT NULL
        AND latitude IS NOT NULL
        AND fire_date IS NOT NULL
        AND latitude >= {region_bounds['lat_min']}
        AND latitude < {region_bounds['lat_max']}
        AND longitude >= {region_bounds['lon_min']}
        AND longitude < {region_bounds['lon_max']}
        ORDER BY fire_date, longitude, latitude
        """

        df = self.client.query(query).to_dataframe()

        if len(df) == 0:
            print(f"NO DATA FOUND")
            return pd.DataFrame()

        df['fire_date'] = pd.to_datetime(df['fire_date'])

        clustered_df = self._cluster_region_data_diagnostic(df, params, region_name)

        if len(clustered_df) > 0:
            clustered_df['region'] = region_name
            return clustered_df
        else:
            return pd.DataFrame()

    def run_diagnostic(self, year):
        """Run diagnostic on all regions"""

        print(f"DIAGNOSTIC FIRE CLUSTERING for {year}")
        print("Using more permissive parameters to identify data loss points")

        regions = {
            'Pacific_West': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -125.0, 'lon_max': -115.0},
            'Mountain_West': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -115.0, 'lon_max': -105.0},
            'Great_Plains': {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -105.0, 'lon_max': -95.0},
            'South_Texas': {'lat_min': 25.0, 'lat_max': 32.0, 'lon_min': -105.0, 'lon_max': -95.0},
            'South_Central': {'lat_min': 25.0, 'lat_max': 37.0, 'lon_min': -95.0, 'lon_max': -85.0},
            'Midwest': {'lat_min': 37.0, 'lat_max': 49.0, 'lon_min': -95.0, 'lon_max': -85.0},
            'Southeast': {'lat_min': 25.0, 'lat_max': 37.0, 'lon_min': -85.0, 'lon_max': -75.0},
            'Northeast': {'lat_min': 37.0, 'lat_max': 49.0, 'lon_min': -85.0, 'lon_max': -67.0},
            'Florida': {'lat_min': 25.0, 'lat_max': 32.0, 'lon_min': -85.0, 'lon_max': -80.0}
        }

        total_query = f"""
        SELECT COUNT(*) as total_count
        FROM `{self.project_id}.{self.dataset_id}.emission_{year}`
        WHERE longitude IS NOT NULL
        AND latitude IS NOT NULL
        AND fire_date IS NOT NULL
        """

        total_result = self.client.query(total_query).to_dataframe()
        original_count = total_result['total_count'].iloc[0]

        all_results = []

        for region_name, region_bounds in regions.items():
            result = self.process_region_diagnostic(year, region_name, region_bounds)
            if len(result) > 0:
                all_results.append(result)

        if all_results:
            combined_events = pd.concat(all_results, ignore_index=True)
            combined_events = combined_events.drop_duplicates(subset=['id'], keep='first')

            final_count = len(combined_events)
            coverage = final_count / original_count * 100

            print(f"\n{'='*60}")
            print(f"DIAGNOSTIC SUMMARY")
            print(f"Total original records: {original_count:,}")
            print(f"Total final records: {final_count:,}")
            print(f"Overall coverage: {coverage:.1f}%")

            print(f"\nDATA LOSS ANALYSIS BY REGION:")
            for region, losses in self.data_loss_tracking.items():
                print(f"\n{region}:")
                for loss in losses:
                    print(f"  {loss['stage']}: {loss['original']:,} → {loss['remaining']:,} "
                          f"({loss['lost_percent']:.1f}% lost) [{loss['notes']}]")

            return combined_events
        else:
            print("No results found")
            return pd.DataFrame()

def main():
    print("DIAGNOSTIC FIRE CLUSTERING")
    print("Purpose: Identify exactly where we're losing data")
    print("Uses more permissive parameters to minimize data loss")
    print()

    clustering = DiagnosticFireClustering(
        project_id="code-for-planet",
        dataset_id="emission_db"
    )

    year = 2004
    fire_events = clustering.run_diagnostic(year=year)

    if len(fire_events) > 0:
        print(f"\nSuccessfully processed {len(fire_events):,} data points")
        print(f"Found {len(fire_events['fire_event_id'].unique())} fire events")

        if len(fire_events) > 0:
            event_stats = fire_events.groupby('fire_event_id').agg({
                'fire_date': ['min', 'max'],
                'longitude': ['min', 'max'],
                'latitude': ['min', 'max']
            }).reset_index()

            event_stats.columns = ['fire_event_id', 'start_date', 'end_date',
                                  'min_lon', 'max_lon', 'min_lat', 'max_lat']

            event_stats['spatial_extent_km'] = event_stats.apply(
                lambda row: clustering.haversine_distance(row['min_lat'], row['min_lon'],
                                                        row['max_lat'], row['max_lon']), axis=1
            )

            print(f"Max spatial extent: {event_stats['spatial_extent_km'].max():.1f} km")
            print(f"Average spatial extent: {event_stats['spatial_extent_km'].mean():.1f} km")

            large_events = event_stats[event_stats['spatial_extent_km'] > 100]
            print(f"Events >100km: {len(large_events)}")
    else:
        print("No fire events found")

if __name__ == "__main__":
    main()

DIAGNOSTIC FIRE CLUSTERING
Purpose: Identify exactly where we're losing data
Uses more permissive parameters to minimize data loss

DIAGNOSTIC FIRE CLUSTERING for 2004
Using more permissive parameters to identify data loss points

PROCESSING REGION: Pacific_West
Bounds: {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -125.0, 'lon_max': -115.0}
Parameters: 8.0km, 7d, min_samples=4
     Starting with 41,845 records
     Initial clustering: 381 clusters, 257 noise points
     Temporal processing: 41,588 → 40,973 points
     Final result: 643 events, 40,973 points

PROCESSING REGION: Mountain_West
Bounds: {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -115.0, 'lon_max': -105.0}
Parameters: 8.0km, 7d, min_samples=4
     Starting with 38,643 records
     Initial clustering: 250 clusters, 215 noise points
     Temporal processing: 38,428 → 38,428 points
     Final result: 402 events, 38,428 points

PROCESSING REGION: Great_Plains
Bounds: {'lat_min': 32.0, 'lat_max': 49.0, 'lon_min': -105.0, 'lon